# Interpretability Tests and Experiments

Going to do some prelim tests to show case mech interp techniques, mathematical unpinnings of said techniques and how it applies to the larger project.


In [ ]:
# imports
import torch
import torch.nn.functional as F 
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on: {device}")